In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
cholesterol = pd.read_csv(r'/Users/liuyu/Downloads/cholesterol.csv', index_col=0)
cholesterol

,trt,response
1,1time,3.8612
2,1time,10.3868
3,1time,5.9059
4,1time,3.0609
5,1time,7.7204
6,1time,2.7139
7,1time,4.9243
8,1time,2.3039
9,1time,7.5301
10,1time,9.4123


In [4]:
# 各组样本大小
cholesterol.groupby('trt').count()

,response
trt,
1time,10
2times,10
4times,10
drugD,10
drugE,10


In [6]:
# 各组均值
cholesterol.groupby('trt').mean()

,response
trt,
1time,5.78197
2times,9.22497
4times,12.37478
drugD,15.36117
drugE,20.94752


In [7]:
# 各组标准差
cholesterol.groupby('trt').std()

,response
trt,
1time,2.878113
2times,3.483054
4times,2.923119
drugD,3.454636
drugE,3.345003


In [9]:
# 检验组间差异
# statsmodels库调用aov函数可用于执行方差分析
model = smf.ols('response ~ C(trt)', data=cholesterol).fit()
model

In [10]:
# sum_sq: 
#   1) C(trt) 组间平方和 方差error sum of square （SSB）误差平方和，各组均值与总均值的误差平方和，反映各样本均值之间的差异程度 
#   2）Residual 组内平方和，各组数据与其均值的误差平方和，反映每个样本各观测值得离散情况
# mean_sq: mean squared error 均方误差
# df: 自由度
# F值：检验组间方差和组内方差的比率
# PR(>F): p值，用于判断组间差异是否显著。如果p值小于显著性水平（通常是0.05），则可以拒绝原假设（即组间均值相等）。
anova_table = sm.stats.anova_lm(model, type=2)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(trt),4.0,1351.369014,337.842253,32.432826,9.818516e-13
Residual,45.0,468.750438,10.416676,NaN,NaN


In [ ]:
# 解释sm.stats.anova_lm(model, type=2)
# type=1: Type I ANOVA(顺序方差分析)，假设每个因子在模型中的顺序是有意义的，计算每个因子时忽略其他因子对模型的贡献
# type=2: Type II ANOVA(平衡方差分析)，假设所有因子是平衡的，计算每个因子的效果时考虑其他因子，但不包括因子之间的交互作用
# type=3: Type III ANOVA(全模型方差分析)，假设模型中的所有因子和交互作用都是相关的，计算每个因子的效果时考虑其他因子和交互作用

In [11]:
cholesterol.response.mean()

12.738081999999999

In [29]:
((cholesterol.groupby('trt').mean() - cholesterol.response.mean()).apply(np.square) * 10).sum()

response    1351.369014
dtype: float64

In [24]:
tmp = cholesterol.merge(cholesterol.groupby('trt').mean().reset_index(), on='trt', how='left')

In [27]:
np.square(tmp.response_x - tmp.response_y).sum()

468.750438055